In [12]:
import sys
from pathlib import Path
import time

# Set manually based on your repo's structure
repo_root = Path.cwd().parent  # or Path.cwd() if you're in the repo root
sys.path.append(str(repo_root))

# ViventisManager Example Notebook

In [13]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os

In [3]:
# --- Set input data path ---
main_path = "/g/mif/people/gritti/code/pymif_test_data"
dataset_name = "viventis"
viventis_path = f"{main_path}/{dataset_name}"  # Update this to your real path


In [4]:
# --- Initialize manager ---
dataset = mm.ViventisManager(path=viventis_path)

In [5]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

Axes: tczyx
Size: [(5, 2, 81, 2304, 2304)]
Scales: [(1.0, 0.173, 0.173)]
Chunk size: (1, 1, 16, 256, 256)
Channels: ['Hoechst', 'FM4-64']
Colors: [16711935, -16776961]


SIZE: [(5, 2, 81, 2304, 2304)]
SCALES: [(1.0, 0.173, 0.173)]
UNITS: ('µm', 'µm', 'µm')
TIME_INCREMENT: 900.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Hoechst', 'FM4-64']
CHANNEL_COLORS: [16711935, -16776961]
DTYPE: uint16
PLANE_FILES: {(0, 0): 't0001_Hoechst.tif', (0, 1): 't0001_FM4-64.tif', (1, 0): 't0002_Hoechst.tif', (1, 1): 't0002_FM4-64.tif', (2, 0): 't0003_Hoechst.tif', (2, 1): 't0003_FM4-64.tif', (3, 0): 't0004_Hoechst.tif', (3, 1): 't0004_FM4-64.tif', (4, 0): 't0005_Hoechst.tif', (4, 1): 't0005_FM4-64.tif'}
AXES: tczyx


In [6]:
dataset.data[0]

dask.array<rechunk-merge, shape=(5, 2, 81, 2304, 2304), dtype=uint16, chunksize=(1, 1, 16, 256, 256), chunktype=numpy.ndarray>

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# NOT RECOMMENDED FOR VERY LARGE DATASETS!
# To visualize lower resolution, build the pyramid and use the start_level argument (last cell).
# To visualize the whole dataset at full resolution, convert data into ome-zarr

viewer = dataset.visualize()

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels, generate lower resolution and load it into memory
viewer = dataset.visualize( start_level = 2, in_memory = True )
# NOTE: This fails because the dataset only has 1 resolution layer

In [ ]:
# --- Build pyramid if not already ---
dataset.build_pyramid(num_levels=5, downscale_factor=2)

# --- Optional: Write to OME-Zarr format ---
output_path = f"{main_path}/{dataset_name}.zarr"
manager.write(output_zarr_path)

In [ ]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

# Now we use the ZarrManager to load the new data and display it

In [14]:
input_zarr_path = f"{main_path}/{dataset_name}.zarr"

In [15]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
# Here, we use the same chunks.
zarr_dataset = mm.ZarrManager(path=input_zarr_path, chunks=(1,1,32,2048,2048))

In [16]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

Axes: tczyx
Size: [(5, 2, 81, 2304, 2304), (5, 2, 41, 1152, 1152), (5, 2, 21, 576, 576)]
Scales: [(1.0, 0.173, 0.173), (2.0, 0.346, 0.346), (4.0, 0.692, 0.692)]
Chunk size: (1, 1, 32, 2048, 2048)
Channels: ['Hoechst', 'FM4-64']
Colors: [16711935, 255]


SIZE: [(5, 2, 81, 2304, 2304), (5, 2, 41, 1152, 1152), (5, 2, 21, 576, 576)]
SCALES: [(1.0, 0.173, 0.173), (2.0, 0.346, 0.346), (4.0, 0.692, 0.692)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 900.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Hoechst', 'FM4-64']
CHANNEL_COLORS: [16711935, 255]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx


In [17]:
zarr_dataset.data[0]

dask.array<rechunk-merge, shape=(5, 2, 81, 2304, 2304), dtype=uint16, chunksize=(1, 1, 32, 2048, 2048), chunktype=numpy.ndarray>

In [18]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster but not interactive.
start = time.time()
viewer = zarr_dataset.visualize()
print(f"{time.time()-start}")

9.301416635513306


In [20]:
start = time.time()
viewer = zarr_dataset.visualize(                        
                        start_level = 0, 
                        stop_level = 1, 
                        in_memory = True 
)
print(f"{time.time()-start}")

25.292306900024414
